IMPORTING REQUIRED LIBRARIES

In [3]:
import pandas as pd

READING CSV

In [4]:
providers = pd.read_csv('/Users/dathril/Desktop/localfoodwaste_managementproject/providers_data (2).csv')
receivers = pd.read_csv('/Users/dathril/Desktop/localfoodwaste_managementproject/receivers_data (1).csv')
food_listings = pd.read_csv('/Users/dathril/Desktop/localfoodwaste_managementproject/food_listings_data.csv')
claims = pd.read_csv('/Users/dathril/Desktop/localfoodwaste_managementproject/claims_data (1).csv')

DATA VALUES

In [5]:
providers.head()

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577


In [6]:
receivers.head()

,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295
1,2,Laurie Ramos,Individual,Lewisburgh,761.042.1570
2,3,Ashley Mckee,NGO,South Randalltown,691-023-0094x856
3,4,Erika Rose,NGO,South Shaneville,8296491111
4,5,John Romero,Individual,Bakerport,067.491.0154


In [7]:
claims.head()

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24
2,3,626,492,Completed,3/21/2025 0:59
3,4,61,933,Cancelled,3/4/2025 9:08
4,5,345,229,Pending,3/14/2025 15:17


In [8]:
food_listings.head()

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,3/24/2025,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,3/28/2025,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,3/16/2025,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,3/19/2025,279,Restaurant,Garciaport,Vegan,Dinner


CHECKING FOR NULL VALUES

In [9]:
print(providers.isnull().sum())

Provider_ID    0
Name           0
Type           0
Address        0
City           0
Contact        0
dtype: int64


In [10]:
print(receivers.isnull().sum())

Receiver_ID    0
Name           0
Type           0
City           0
Contact        0
dtype: int64


In [11]:
print(claims.isnull().sum())

Claim_ID       0
Food_ID        0
Receiver_ID    0
Status         0
Timestamp      0
dtype: int64


In [12]:
print(food_listings.isnull().sum())

Food_ID          0
Food_Name        0
Quantity         0
Expiry_Date      0
Provider_ID      0
Provider_Type    0
Location         0
Food_Type        0
Meal_Type        0
dtype: int64


ALL THE COLUMNS ARE FULL NO NULL VALUES SO WE CAN GO AHEAD

connecting to the database

In [13]:
import sqlite3
conn = sqlite3.connect('food_donation.db')
cursor = conn.cursor()


In [14]:
providers.to_sql("providers",conn,if_exists='replace',index= False)
receivers.to_sql("receivers",conn,if_exists='replace',index=False)
claims.to_sql("claims",conn,if_exists='replace',index=False)
food_listings.to_sql("food_listings",conn,if_exists='replace',index=False)

1000

How many food providers and receivers are there in each city?



In [15]:
query = """
SELECT city, COUNT(*) AS total_providers
FROM providers
GROUP BY city;



"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,City,total_providers
0,Adambury,1
1,Adamsview,1
2,Adamsville,1
3,Aguirreville,1
4,Alexanderchester,1


In [16]:
query = """
SELECT city, COUNT(*) AS total_receivers
FROM receivers
GROUP BY city;



"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,City,total_receivers
0,Aaronshire,1
1,Adamland,1
2,Aguilarbury,1
3,Aguilarstad,1
4,Alexanderbury,1


2.Which type of food provider (restaurant, grocery store, etc.) contributes the most food?

In [17]:
query = """
SELECT p.type, SUM(f.quantity) AS total_food_contributed
FROM providers p
JOIN food_listings f ON p.provider_id = f.provider_id
GROUP BY p.type
ORDER BY total_food_contributed DESC
LIMIT 1;

"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,Type,total_food_contributed
0,Restaurant,6923


3.What is the contact information of food providers in a specific city?


In [18]:
query = """
SELECT name, contact
FROM providers
WHERE city = 'Ahmedabad';
"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,Name,Contact


4.Which receivers have claimed the most food?


In [19]:
query = """
SELECT r.name, SUM(f.quantity) AS total_claimed
FROM receivers r
JOIN claims c ON r.receiver_id = c.receiver_id
JOIN food_listings f ON c.food_id = f.food_id
GROUP BY r.name
ORDER BY total_claimed DESC
LIMIT 5;
"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,Name,total_claimed
0,Matthew Webb,191
1,Donald Caldwell,174
2,Anthony Garcia,162
3,Scott Hunter,157
4,Jennifer Nelson,139


5.What is the total quantity of food available from all providers?


In [20]:
query = """
SELECT Location,COUNT(*) AS total_available_food
FROM food_listings
"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,Location,total_available_food
0,South Kellyville,1000


6.Which city has the highest number of food listings?


In [21]:
query = """
SELECT Location,COUNT(*) AS CITY_COUNT
FROM food_listings
GROUP BY Location
ORDER BY CITY_COUNT DESC
LIMIT 1;

"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,Location,CITY_COUNT
0,South Kathryn,6


7.What are the most commonly available food types?


In [22]:
query = """
SELECT food_type,Count(*) AS FOOD_TYPE_COUNT
FROM food_listings
GROUP BY food_type
ORDER BY FOOD_TYPE_COUNT DESC
"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,Food_Type,FOOD_TYPE_COUNT
0,Vegetarian,336
1,Vegan,334
2,Non-Vegetarian,330


8.How many food claims have been made for each food item?



In [23]:
query = """
SELECT food_type,Count(claim_id)
FROM food_listings f
JOIN claims c ON f.food_id = c.food_id
GROUP BY food_type;
"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,Food_Type,Count(claim_id)
0,Non-Vegetarian,331
1,Vegan,319
2,Vegetarian,350


9.Which provider has had the highest number of successful food claims?


In [27]:
query = """
SELECT p.provider_id,c.Status
FROM claims c
JOIN food_listings f ON p.provider_id = f.provider_id
JOIN providers p ON p.provider_id = f.provider_id
WHERE Status ="successful"
GROUP by status;
"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,Provider_ID,Status


10.What percentage of food claims are completed vs. pending vs. canceled?


In [28]:
query = """
SELECT status,COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) AS percentage
FROM claims
GROUP BY status;
"""
df_joined = pd.read_sql_query(query,conn)
display(df_joined.head())

,Status,percentage
0,Cancelled,33.6
1,Completed,33.9
2,Pending,32.5


11.What is the average quantity of food claimed per receiver?


In [29]:
query = """
SELECT r.name, AVG(f.quantity) AS avg_claimed
FROM claims c
JOIN receivers r ON c.receiver_id = r.receiver_id
JOIN food_listings f ON c.food_id = f.food_id
GROUP BY r.name;
"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,Name,avg_claimed
0,Aaron Keller,39.000000
1,Aaron Rios,21.000000
2,Aaron Scott,45.000000
3,Abigail Crawford,25.666667
4,Adam Browning,5.000000


12.Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?


In [30]:
query = """
SELECT f.meal_type, SUM(f.quantity) AS total_claimed
FROM claims c
JOIN food_listings f ON c.food_id = f.food_id
GROUP BY f.meal_type
ORDER BY total_claimed DESC
LIMIT 3;
"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,Meal_Type,total_claimed
0,Breakfast,7363
1,Lunch,6373
2,Snacks,6336


13.What is the total quantity of food donated by each provider?


In [31]:
query = """
SELECT p.name, SUM(f.quantity) AS total_donated
FROM providers p
JOIN food_listings f ON p.provider_id = f.provider_id
GROUP BY p.name
ORDER BY total_donated DESC;
"""
df_joined = pd.read_sql_query(query,conn)
df_joined.head()

,Name,total_donated
0,Miller Inc,217
1,Barry Group,179
2,"Evans, Wright and Mitchell",158
3,Smith Group,150
4,Campbell LLC,145


In [32]:
!pip install streamlit

  Using cached streamlit-1.49.1-py3-none-any.whl.metadata (9.5 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached narwhals-2.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2025.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached

In [33]:
import streamlit as st
import pandas as pd
import sqlite3

# -----------------------------
# Connect to SQLite DB
# -----------------------------
conn = sqlite3.connect("food_donation.db")

st.set_page_config(page_title="🍴 Local Food Wastage Management", layout="wide")

st.title("🍴 Local Food Wastage Management System")
st.markdown("Analyze and manage surplus food distribution using SQL + Streamlit")

# -----------------------------
# Sidebar Navigation
# -----------------------------
menu = ["Providers", "Receivers", "Food Listings", "Claims", "Insights"]
choice = st.sidebar.radio("Navigate", menu)

# -----------------------------
# Providers
# -----------------------------
if choice == "Providers":
    st.subheader("📌 Providers Data")
    df = pd.read_sql_query("SELECT * FROM providers;", conn)
    st.dataframe(df)

# -----------------------------
# Receivers
# -----------------------------
elif choice == "Receivers":
    st.subheader("📌 Receivers Data")
    df = pd.read_sql_query("SELECT * FROM receivers;", conn)
    st.dataframe(df)

# -----------------------------
# Food Listings
# -----------------------------
elif choice == "Food Listings":
    st.subheader("📌 Food Listings Data")
    df = pd.read_sql_query("SELECT * FROM food_listings;", conn)
    st.dataframe(df)

# -----------------------------
# Claims
# -----------------------------
elif choice == "Claims":
    st.subheader("📌 Claims Data")
    df = pd.read_sql_query("SELECT * FROM claims;", conn)
    st.dataframe(df)

# -----------------------------
# Insights (SQL Queries)
# -----------------------------
elif choice == "Insights":
    st.subheader("📊 Analysis & Insights")

    # 1. Providers per city
    q1 = "SELECT city, COUNT(*) AS total_providers FROM providers GROUP BY city;"
    st.write("**Food Providers per City**")
    st.dataframe(pd.read_sql_query(q1, conn))

    # 2. Receivers per city
    q2 = "SELECT city, COUNT(*) AS total_receivers FROM receivers GROUP BY city;"
    st.write("**Food Receivers per City**")
    st.dataframe(pd.read_sql_query(q2, conn))

    # 3. Provider type contribution
    q3 = """
    SELECT provider_type, SUM(quantity) AS total_contribution
    FROM food_listings
    GROUP BY provider_type
    ORDER BY total_contribution DESC;
    """
    st.write("**Food Contribution by Provider Type**")
    st.bar_chart(pd.read_sql_query(q3, conn).set_index("provider_type"))

    # 4. Receivers with most claims
    q4 = """
    SELECT r.name, SUM(f.quantity) AS total_claimed
    FROM receivers r
    JOIN claims c ON r.receiver_id = c.receiver_id
    JOIN food_listings f ON c.food_id = f.food_id
    GROUP BY r.name
    ORDER BY total_claimed DESC
    LIMIT 5;
    """
    st.write("**Top 5 Receivers by Total Food Claimed**")
    st.dataframe(pd.read_sql_query(q4, conn))

    # 5. Total quantity available
    q5 = "SELECT SUM(quantity) AS total_available FROM food_listings;"
    st.metric("Total Available Food", int(pd.read_sql_query(q5, conn)["total_available"][0]))

    # 6. City with highest food listings
    q6 = """
    SELECT city, COUNT(*) AS total_listings
    FROM food_listings
    GROUP BY city
    ORDER BY total_listings DESC
    LIMIT 1;
    """
    st.write("**City with Highest Food Listings**")
    st.dataframe(pd.read_sql_query(q6, conn))

    # 7. Most common food types
    q7 = """
    SELECT food_type, COUNT(*) AS count_food
    FROM food_listings
    GROUP BY food_type
    ORDER BY count_food DESC;
    """
    st.write("**Most Common Food Types**")
    st.bar_chart(pd.read_sql_query(q7, conn).set_index("food_type"))

    # 8. Claims per food item
    q8 = """
    SELECT f.food_name, COUNT(c.claim_id) AS total_claims
    FROM claims c
    JOIN food_listings f ON c.food_id = f.food_id
    GROUP BY f.food_name
    ORDER BY total_claims DESC;
    """
    st.write("**Food Claims per Item**")
    st.dataframe(pd.read_sql_query(q8, conn))

    # 9. Provider with most successful claims
    q9 = """
    SELECT p.name, COUNT(c.claim_id) AS successful_claims
    FROM claims c
    JOIN providers p ON c.provider_id = p.provider_id
    WHERE c.status = 'Completed'
    GROUP BY p.name
    ORDER BY successful_claims DESC
    LIMIT 1;
    """
    st.write("**Provider with Most Successful Claims**")
    st.dataframe(pd.read_sql_query(q9, conn))

    # 10. Claim status percentages
    q10 = """
    SELECT status, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) AS percentage
    FROM claims
    GROUP BY status;
    """
    st.write("**Claim Status Distribution (%)**")
    st.dataframe(pd.read_sql_query(q10, conn))

    # 11. Avg quantity claimed per receiver
    q11 = """
    SELECT r.name, AVG(f.quantity) AS avg_claimed
    FROM claims c
    JOIN receivers r ON c.receiver_id = r.receiver_id
    JOIN food_listings f ON c.food_id = f.food_id
    GROUP BY r.name;
    """
    st.write("**Average Quantity Claimed per Receiver**")
    st.dataframe(pd.read_sql_query(q11, conn))

    # 12. Most claimed meal type
    q12 = """
    SELECT f.meal_type, SUM(f.quantity) AS total_claimed
    FROM claims c
    JOIN food_listings f ON c.food_id = f.food_id
    GROUP BY f.meal_type
    ORDER BY total_claimed DESC
    LIMIT 1;
    """
    st.write("**Most Claimed Meal Type**")
    st.dataframe(pd.read_sql_query(q12, conn))

    # 13. Total food donated by each provider
    q13 = """
    SELECT p.name, SUM(f.quantity) AS total_donated
    FROM providers p
    JOIN food_listings f ON p.provider_id = f.provider_id
    GROUP BY p.name
    ORDER BY total_donated DESC;
    """
    st.write("**Total Quantity Donated by Provider**")
    st.dataframe(pd.read_sql_query(q13, conn))

st.sidebar.markdown("---")
st.sidebar.info("Developed with ❤️ using Python, SQL & Streamlit")


2025-09-14 01:47:54.751 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-14 01:47:54.751 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-14 01:47:54.787 
  command:

    streamlit run /Users/dathril/.pyenv/versions/3.10.8/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-09-14 01:47:54.787 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-14 01:47:54.787 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-14 01:47:54.788 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-14 01:47:54.788 Thread 'MainThread': missing ScriptRunContext! This warning 

DeltaGenerator(_root_container=1, _parent=DeltaGenerator())